## Imports

In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx 
import folium
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString
import json

import random
import os 

## Data

In [35]:
df_links = pd.read_csv("df_links.csv") 
df_distances = pd.read_csv("distance_matrix_from_links.csv") 
df_ville_lignes = pd.read_csv("lines_to_city_map.csv")

In [36]:
df_links.start_node.unique()

array(['Genève', 'Lausanne', 'Fribourg/Freiburg', 'Neuchâtel',
       'Biel/Bienne', 'Bern', 'Thun', 'Olten', 'Aarau', 'Baden',
       'Zürich HB', 'Luzern', 'Zürich Oerlikon', 'Zug', 'Winterthur',
       'St. Gallen', 'Bellinzona', 'Lugano Nord'], dtype=object)

In [41]:
df_links.head()

,Unnamed: 0,start_node,end_node,distance [m]
0,0,Genève,Genève-Aéroport,5918.188
1,1,Genève,Lausanne,60226.630
2,2,Genève,Neuchâtel,124954.163
3,3,Lausanne,Fribourg/Freiburg,65778.350
4,4,Lausanne,Neuchâtel,75087.957


In [37]:
df_distances.head()

,Unnamed: 0,Bern,Basel SBB,Lausanne,Luzern,St. Gallen,Winterthur,Zug,Aarau,Baden,...,Genève-Aéroport,Genève,Neuchâtel,Olten,Thun,Bellinzona,Lugano,Lugano Nord,Zürich Oerlikon,Zürich HB
0,Bern,0.000,-1.0,-1.0,119979.281,-1.0,-1.000,-1.000,-1.000,-1.000,...,-1.0,-1.00,-1.000,65626.280,31264.012,-1.000,-1.0,-1.0,-1.0,-1.0
1,Basel SBB,-1.000,0.0,-1.0,-1.000,-1.0,-1.000,-1.000,-1.000,-1.000,...,-1.0,-1.00,-1.000,39172.397,-1.000,-1.000,-1.0,-1.0,-1.0,-1.0
2,Lausanne,-1.000,-1.0,0.0,-1.000,-1.0,-1.000,-1.000,-1.000,-1.000,...,-1.0,60226.63,75087.957,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.0
3,Luzern,119979.281,-1.0,-1.0,0.000,-1.0,-1.000,28417.295,65019.211,73096.897,...,-1.0,-1.00,-1.000,55768.093,147480.179,176738.373,-1.0,-1.0,-1.0,-1.0
4,St. Gallen,-1.000,-1.0,-1.0,-1.000,0.0,57139.042,-1.000,-1.000,-1.000,...,-1.0,-1.00,-1.000,-1.000,-1.000,-1.000,-1.0,-1.0,-1.0,-1.0


In [40]:
df_ville_lignes.head()

,Unnamed: 0,ville,lignes
0,0,Bern,"['IC1', 'IC6', 'IC8', 'IC61', 'IC81']"
1,1,Basel SBB,"['IC3', 'IC6', 'IC21', 'IC51', 'IC61']"
2,2,Lausanne,"['IC1', 'IC5']"
3,3,Luzern,['IC21']
4,4,St. Gallen,"['IC1', 'IC5']"


## Create a random OD table to start working 

In [30]:

cities = ['Genève-Aéroport', 'Genève', 'Zürich Altstetten',
          'Zürich Hardbrücke', 'Zürich Oerlikon', 'Zürich HB', 'Zürich Enge',
          'Thun', 'Basel SBB', 'Lausanne', 'Winterthur', 'St. Gallen',
          'Luzern', 'Lugano', 'Baden', 'Fribourg/Freiburg', 'Biel/Bienne',
          'Neuchâtel', 'Bellinzona', 'Zug', 'Bern', 'Olten', 'Chur', 'Aarau']

n = len(cities)

np.random.seed(42)  # for reproducibility
od_matrix = np.random.randint(0, 10000, size=(n, n))

np.fill_diagonal(od_matrix, 0)

df_od = pd.DataFrame(od_matrix, index=cities, columns=cities)

df_od.head()

,Genève-Aéroport,Genève,Zürich Altstetten,Zürich Hardbrücke,Zürich Oerlikon,Zürich HB,Zürich Enge,Thun,Basel SBB,Lausanne,...,Baden,Fribourg/Freiburg,Biel/Bienne,Neuchâtel,Bellinzona,Zug,Bern,Olten,Chur,Aarau
Genève-Aéroport,0,860,5390,5191,5734,6265,466,4426,5578,8322,...,5311,5051,6420,1184,4555,3385,6396,8666,9274,2558
Genève,7849,0,2747,9167,9998,189,2734,3005,4658,1899,...,3890,8838,5393,8792,8433,7513,2612,7041,9555,6235
Zürich Altstetten,5486,7099,0,775,8226,3152,1585,3943,7555,3073,...,6873,5675,161,4297,995,7629,9467,1016,7869,6439
Zürich Hardbrücke,7892,6863,7916,0,878,9268,4887,4859,6331,8571,...,64,8006,2568,5463,2027,2695,9687,5258,5618,6736
Zürich Oerlikon,391,5892,3561,6184,0,6278,8392,3104,7215,2454,...,5056,8110,3840,1028,7385,502,6910,9062,6938,4488


## Trials